In [1]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold

from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import auc as calculate_auc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from sklearn.utils import shuffle 
from joblib import load, dump
import numpy as np
import pandas as pd
import os

In [2]:
from chembench import load_data, dataset
from molmap import feature

In [3]:
bitsinfo = feature.fingerprint.Extraction().bitsinfo
fp_types = bitsinfo.Subtypes.unique()
fp_types

array(['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP',
       'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP',
       'MHFP6', 'MAP4'], dtype=object)

In [4]:
from scipy.stats.stats import pearsonr
def r2(y_true, y_pred):
    pcc, _ = pearsonr(y_true,y_pred)
    return pcc[0]**2

def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)  
    return rmse


def PRC_AUC(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

def ROC_AUC(y_true, y_score):
    auc = roc_auc_score(y_true, y_score)
    return auc

In [5]:
esol = dataset.load_ESOL()
lipop = dataset.load_Lipop()
FreeSolv = dataset.load_FreeSolv()
PDBF = dataset.load_PDBF()

datasets = [esol, lipop, FreeSolv] #malaria

total samples: 1128
total samples: 4200
total samples: 642
total samples: 9880


In [6]:
performance = []

for data in datasets:
    for fp_type in fp_types:
        task_name = data.task_name
        print(task_name, fp_type)
        df, induces = load_data(task_name)
        X2 = load('/raid/shenwanxiang/10_FP_effect/tempignore/X2_%s_%s.data' % (task_name, fp_type) )
        n, w, c = X2.sum(axis=-1).shape
        X2 = X2.reshape(n, w*c)
        Y = data.y
        for i, idx in enumerate(induces):
            train_idx, valid_idx, test_idx = idx

            X = X2[train_idx]
            y = Y[train_idx]

            X_valid = X2[valid_idx]
            y_valid = Y[valid_idx]

            X_test = X2[test_idx]
            y_test = Y[test_idx] 

            # Set up possible values of parameters to optimize over
            n_neighbors_list = np.arange(1, 15, 2)
            weights_list =  ['uniform', 'distance']
            res = []
            for n_neighbors in tqdm(n_neighbors_list, ascii=True):
                for weights in weights_list:
                    clf = KNeighborsRegressor(n_neighbors=n_neighbors, weights = weights)
                    clf.fit(X, y)
                    score = clf.score(X_valid, y_valid)
                    res.append([n_neighbors, weights, score])

            dfr = pd.DataFrame(res, columns = ['n_neighbors', 'weights', 'score'])
            gidx = dfr['score'].idxmax()
            best_params = dfr.iloc[gidx].to_dict()
            best_params.pop('score')
            best_params

            clf = KNeighborsRegressor(**best_params)
            clf.fit(X, y, )

            test_r2 = r2(y_test, clf.predict(X_test))
            test_rmse = rmse(y_test, clf.predict(X_test))
            results = {"task_name":task_name, 'fp_type':fp_type,"split-time":i, "test_rmse":test_rmse , "test_r2": test_r2}

            performance.append(results)

  0%|          | 0/14 [00:00<?, ?it/s]

ESOL MorganFP
loading dataset: ESOL number of split times: 3


  0%|          | 0/14 [00:00<?, ?it/s]

ESOL RDkitFP
loading dataset: ESOL number of split times: 3


  0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

FreeSolv MHFP6
loading dataset: FreeSolv number of split times: 3


  0%|          | 0/14 [00:00<?, ?it/s]

FreeSolv MAP4
loading dataset: FreeSolv number of split times: 3


100%|##########| 14/14 [00:04<00:00,  3.20it/s]


In [8]:
pd.DataFrame(performance).to_csv('./knn_regression.csv')

In [16]:
pd.DataFrame(performance).groupby(['task_name','fp_type'])['test_r2'].apply(lambda x:x.mean())

task_name  fp_type      
ESOL       AtomPairFP       0.723782
           AvalonFP         0.703896
           EstateFP         0.518462
           MACCSFP          0.703028
           MAP4             0.570213
           MHFP6            0.642149
           MorganFP         0.524776
           PharmacoErGFP    0.560204
           PharmacoPFP      0.695061
           PubChemFP        0.730567
           RDkitFP          0.676306
           TorsionFP        0.508625
FreeSolv   AtomPairFP       0.684389
           AvalonFP         0.805264
           EstateFP         0.781209
           MACCSFP          0.818480
           MAP4             0.352572
           MHFP6            0.745899
           MorganFP         0.501347
           PharmacoErGFP    0.767449
           PharmacoPFP      0.608869
           PubChemFP        0.806931
           RDkitFP          0.761272
           TorsionFP        0.398211
Lipop      AtomPairFP       0.436795
           AvalonFP         0.390984
           Es